In [ ]:
pip install finance-datareader

In [ ]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#종목 코드 받아오고 제출할 df 형식 가져오기 
path = './'
stock_list = pd.read_csv("/content/drive/MyDrive/stock/stock_list.csv")
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6)) #파에 0을 체워주는 함수 
sample_submission = pd.read_csv("/content/drive/MyDrive/stock/sample_submission.csv")


In [ ]:
#날짜별 주식 값 가져오기 
start_date = '20210104'
end_date = '20211126'
start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
print(Business_days)
data = Business_days.copy()
stock_list['종목코드'].values.sort()
for code in tqdm(stock_list['종목코드'].values):
    fdr_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(data,fdr_data, how='outer', on='Date')
    # data.Close = data.Close.ffill()
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)
columns = list(sample_submission.columns[1:])
columns = ['Day'] + [str(x).zfill(6) for x in columns]
data.columns = columns
data.to_csv("/content/drive/MyDrive/stock/stockPrice.csv", index=False)
price1126= pd.read_csv('/content/drive/MyDrive/stock/stockPrice.csv')
price1022=price1126.iloc[:210,:]

          Date
0   2021-01-04
1   2021-01-05
2   2021-01-06
3   2021-01-07
4   2021-01-08
..         ...
230 2021-11-22
231 2021-11-23
232 2021-11-24
233 2021-11-25
234 2021-11-26

[235 rows x 1 columns]


100%|██████████| 370/370 [05:17<00:00,  1.17it/s]


In [ ]:
price1126= pd.read_csv('/content/drive/MyDrive/stock/stockPrice.csv')
price1022=price1126.iloc[:210,:]
price1025_29=price1126.iloc[210:215,:]
price1029=price1126.iloc[:215,:]
price1119=price1126.iloc[:230,:]
price1122_26=price1126.iloc[230:235,:]

###요일로 선형회귀

In [ ]:

def day_linear(start_date,end_date,price):
  data2 = price
  Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

  model = LinearRegression()
  #학습
  sample_submission = pd.read_csv("/content/drive/MyDrive/stock/sample_submission.csv")

  model = LinearRegression()
  for code in tqdm(stock_list['종목코드'].values): #tqdm 진행률 
    data = pd.DataFrame(data2.loc[:,code]).reset_index(drop=False,inplace=False)
    data.columns=["Date","Close"]
    data=pd.concat([Business_days,data["Close"]],axis=1)
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill() #null 값이 있으면 인덱스 앞이나 뒤값으로 채운다 
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    
    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 10월 22일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 10월 29일까지의 데이터를 학습한다.
    y_0 = y[:,0]
    y_1 = y[:,1]
    y_2 = y[:,2]
    y_3 = y[:,3]
    y_4 = y[:,4]
    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy()
    predictions = []
    for y_value in y_values :
        model.fit(x,y_value)
        prediction = model.predict(np.expand_dims(x_public,0))
        predictions.append(prediction[0])
    sample_submission.loc[:,code] = predictions * 2
  sample_submission.isna().sum().sum()
  sample_submission1=sample_submission.iloc[0:5,:].drop("Day",axis=1)
  return sample_submission1


### ARIMA

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive, files
drive.mount('/content/drive')
import FinanceDataReader as fdr
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

def ari(start_date,end_date,price):
  sub = pd.read_csv("/content/drive/MyDrive/stock/sample_submission.csv")
  data2 = price
  Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
  test_data=[]
  data2=data2.drop("Day",axis=1)
  for i in tqdm(range(len(data2.columns))):
    test=data2.iloc[:,i]
    model = ARIMA(test.values, order=(0,1,1))
    model_fit = model.fit(trend='c', full_output=True, disp=True)
    forecast_data = model_fit.forecast(steps=5)
    test_data.append(forecast_data[0])


  sample_submission2 = pd.read_csv("/content/drive/MyDrive/stock/sample_submission.csv").iloc[0:5,:].drop("Day",axis=1)
  for i in range(stock_list["종목코드"].size):
    sample_submission2.iloc[:,i] = test_data[i]
  return sample_submission2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###5이평선 선형 회귀

In [ ]:
def rolling_linear(start_date,end_date,price):

  data2 = price
  Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
  sample_submission3 = pd.read_csv("/content/drive/MyDrive/stock/sample_submission.csv").iloc[0:5,:].drop("Day",axis=1)
  for ss in [5]:
    model = LinearRegression()
    answer_m5=[]
    for code in tqdm(sample_submission3.columns.values):
      data = pd.DataFrame(data2.loc[:,code]).reset_index(drop=False,inplace=False)
      data.columns=["Date","Close"]
      data=pd.concat([Business_days,data["Close"]],axis=1)
      data['weekday'] = data.Date.apply(lambda x : x.weekday())
      data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
      data.Close = data.Close.ffill()

      data['ma5']=data['Close'].rolling(ss).mean()#지수 이동평균 최근값에 가중치를 두면서 이동평균계산 
      data['ma20']=data['Close'].rolling(ss).mean()
      # print(data["Close"].tolist())
      xx=data[["ma5","Close"]]
      x_train=xx.iloc[5:-1,:]["ma5"].values.reshape((-1,1))  #전날의 이평선 가격 
      x_pred = xx.iloc[6:,:]["Close"].values.reshape((-1,1)) #오늘의 종가
    
      model.fit(x_train,x_pred)
      x1=xx.iloc[5:,:]
      y1=xx.iloc[5:,:]["Close"]
      y1["one"]=model.predict([[x1.iloc[-1,0]]])[0][0] #어제 이평선 가격으로 오늘 종가 예측
      y1=pd.DataFrame(y1, columns=['Close'])
      y1['ma5']=y1["Close"].rolling(ss).mean()

      y2=y1["Close"]
      y2["two"]=model.predict([[y1.iloc[-1,0]]])[0][0]
      y2=pd.DataFrame(y2, columns=['Close'])
      y2['ma5']=y2["Close"].rolling(ss).mean()

      y3=y2["Close"]
      y3["two"]=model.predict([[y2.iloc[-1,0]]])[0][0]
      y3=pd.DataFrame(y3, columns=['Close'])
      y3['ma5']=y3["Close"].rolling(ss).mean()

      y4=y3["Close"]
      y4["two"]=model.predict([[y3.iloc[-1,0]]])[0][0]
      y4=pd.DataFrame(y4, columns=['Close'])
      y4['ma5']=y4["Close"].rolling(ss).mean()

      y5=y4["Close"]
      y5["two"]=model.predict([[y4.iloc[-1,0]]])[0][0]
      y5=pd.DataFrame(y5, columns=['Close'])
      y5['ma5']=y5["Close"].rolling(ss).mean()

      answer_m5.append(y5["Close"].iloc[-5:].tolist())
    for i in range(stock_list["종목코드"].size):
      sample_submission3.iloc[:,i] = answer_m5[i]
  return sample_submission3

In [ ]:
#10월 22일 까지의 데이터로 10월25~29 예측
start_date = '20210104'
end_date = '20211022'
linear_sb= day_linear(start_date,end_date,price1022)
ari_sb= ari(start_date,end_date,price1022)
roll_sb= rolling_linear(start_date,end_date,price1022)

# 종목 각각의 오차 평균을 알고리즘 별로 리스트에 저장 
total_list = [[0 for _ in range(3)] for _ in range(370)] #370
pred= price1025_29.drop("Day",axis=1).reset_index(drop=True)
per =abs(pred -linear_sb)/pred
per1= per.mean()
print(per.mean().sum()*100/370)
for i in range(370):
    total_list[i][0]=per1[i]


per =abs(pred -ari_sb)/pred
per1= per.mean()
print(per.mean().sum()*100/370)
for i in range(370):
    total_list[i][1]=per1[i]


per =abs(pred -roll_sb)/pred
per1= per.mean()
print(per.mean().sum()*100/370)
for i in range(370):
    total_list[i][2]=per1[i]

#각각의 종목을 가장 오차 평균이 낮은 알고리즘에 넣어 10월 29일까지를 넣어서 11월 1~5일 예측
start_date = '20210104'
end_date = '20211029'
linear_sb= day_linear(start_date,end_date,price1029)
ari_sb= ari(start_date,end_date,price1029)
roll_sb= rolling_linear(start_date,end_date,price1029)

li= []
for i in range(370):
  li.append(total_list[i].index(min(total_list[i])))
pos1 = np.where(np.array(li) == 0)[0]
pos2 = np.where(np.array(li) == 1)[0]
pos3= np.where(np.array(li) == 2)[0]
a1=linear_sb.iloc[:,pos1]
a2=ari_sb.iloc[:,pos2]
a3=roll_sb.iloc[:,pos3]
result3 = pd.concat([a1,a2,a3],axis=1)
result3.sort_index(axis = 1,inplace=True)
result3.to_csv("/content/drive/MyDrive/stock/mission.csv", index=False)


 11%|█         | 41/370 [00:01<00:08, 36.88it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
 95%|█████████▌| 352/370 [00:11<00:00, 32.45it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
100%|██████████| 370/370 [00:09<00:00, 37.55it/s]


4.9823492197848
2.8608485097016048
3.541556625864316


 23%|██▎       | 84/370 [00:02<00:08, 34.83it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
100%|██████████| 370/370 [00:08<00:00, 41.23it/s]


In [ ]:
#11월 19일 까지의 데이터로 11월22~26 예측
start_date = '20210104'
end_date = '20211119'
linear_sb= day_linear(start_date,end_date,price1119)
ari_sb= ari(start_date,end_date,price1119)
roll_sb= rolling_linear(start_date,end_date,price1119)

# 종목 각각의 11월 22~26주식 가격의 오차 평균을 알고리즘 별로 리스트에 저장 
total_list = [[0 for _ in range(3)] for _ in range(370)] #370
pred= price1122_26.drop("Day",axis=1).reset_index(drop=True)
per =abs(pred -linear_sb)/pred
per1= per.mean()
print(per.mean().sum()*100/370)
for i in range(370):
    total_list[i][0]=per1[i]


per =abs(pred -ari_sb)/pred
per1= per.mean()
print(per.mean().sum()*100/370)
for i in range(370):
    total_list[i][1]=per1[i]


per =abs(pred -roll_sb)/pred
per1= per.mean()
print(per.mean().sum()*100/370)
for i in range(370):
    total_list[i][2]=per1[i]

#각각의 종목을 가장 오차 평균이 낮은 알고리즘에 넣어 11월 26일까지를 넣어서 11월 29~12~3일 예측 
start_date = '20210104'
end_date = '20211126'
linear_sb= day_linear(start_date,end_date,price1126)
ari_sb= ari(start_date,end_date,price1126)
roll_sb= rolling_linear(start_date,end_date,price1126)

li= []
for i in range(370):
  li.append(total_list[i].index(min(total_list[i])))
pos1 = np.where(np.array(li) == 0)[0]
pos2 = np.where(np.array(li) == 1)[0]
pos3= np.where(np.array(li) == 2)[0]
a1=linear_sb.iloc[:,pos1]
a2=ari_sb.iloc[:,pos2]
a3=roll_sb.iloc[:,pos3]
result4 = pd.concat([a1,a2,a3],axis=1)
result4.sort_index(axis = 1,inplace=True)
result4.to_csv("/content/drive/MyDrive/stock/mission2.csv", index=False)


  6%|▌         | 21/370 [00:00<00:08, 43.60it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
100%|██████████| 370/370 [00:08<00:00, 41.40it/s]


6.234261468662508
3.4841401469669577
4.614885140126459


100%|██████████| 370/370 [00:08<00:00, 41.13it/s]


In [ ]:
mission1_5 = pd.read_csv('/content/drive/MyDrive/stock/mission.csv')
mission29_1 =pd.read_csv('/content/drive/MyDrive/stock/mission2.csv')
final=pd.concat([mission1_5,mission29_1])
final["Day"]=["2021-11-01","2021-11-02","2021-11-03","2021-11-04","2021-11-05","2021-11-29","2021-11-30","2021-12-01","2021-12-02","2021-12-03"]
final.index= final["Day"]
final.drop("Day",axis=1,inplace=True)
final.reset_index(inplace=True)
final.to_csv("/content/drive/MyDrive/stock/final_mission.csv", index=False)
final

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27479.842216,36138.095309,59798.209178,142016.422696,95225.613773,16713.587376,46891.672887,84056.764192,98222.613725,720762.814054,50361.126651,231319.857832,33663.202372,54667.818208,17079.226698,2257.003263,28298.159917,5106.851131,333177.000637,51600.000000,12987.616025,31551.517476,9434.864767,7680.000000,30391.464715,145654.518592,45864.846980,85261.828596,44461.530281,248169.299796,288500.000000,82277.460899,103123.833092,33466.540522,29123.987871,34984.926227,159500.000000,207767.217522,97422.884594,...,40294.094097,106518.185307,283622.808353,12967.648389,174143.743504,179832.868481,7270.000000,7002.958153,35600.000000,13369.888519,79976.672599,50466.661947,50396.668798,25548.855473,286913.279816,599051.747384,711695.030187,21189.389298,52805.779739,71454.758990,27200.000000,119000.000000,37042.570458,13267.648631,3283.739239,36169.435821,18950.000000,65449.124033,94461.628037,5700.000000,46400.000000,52248.763527,82314.402539,36026.351168,25819.107026,51853.261707,335831.067919,264200.000000,27109.877584,17078.431964
1,2021-11-02,27541.090315,36207.635430,59708.022261,141892.590915,96465.097047,16724.335040,46729.289709,84285.364662,97784.823214,719278.675955,50422.266838,231544.156897,33683.239063,55269.834105,17118.338850,2261.043533,28367.731617,5107.016065,333803.168861,50600.000000,12913.316972,31448.160079,9443.896530,7730.000000,30444.957615,145783.023262,47954.766425,85402.015516,44476.988401,248213.692321,284000.000000,82312.001469,103252.805055,33603.656818,29008.077494,34975.580426,158000.000000,207769.553974,97449.987396,...,41181.591587,106502.224256,284050.378446,12899.428591,173999.891121,179439.359135,7220.000000,6998.222654,35100.000000,13366.738991,80134.616525,50548.283437,50279.497980,25544.916607,287511.410657,600846.139908,714302.506823,21142.035163,52693.036748,71583.730952,26900.000000,118000.000000,37087.228199,13285.125366,3233.619442,36625.287961,18950.000000,66268.822658,94181.721488,5660.000000,49200.000000,52229.721880,84160.171715,35994.341816,25693.033315,52211.959579,336665.553900,263800.000000,27106.376844,17057.871214
2,2021-11-03,27602.338413,35963.989673,59617.835344,141768.759133,96891.585430,16735.082703,46566.906532,84293.033940,97999.656839,711611.180447,50483.407024,231768.455963,33703.275755,55239.146521,17157.451003,2265.083803,28447.954048,5107.180999,334429.337085,49600.000000,12839.017920,31344.802682,9387.676928,7850.000000,30325.218738,145911.527932,46983.363996,85542.202435,44492.446521,248258.084847,283500.000000,82346.542039,103381.777017,33452.259459,28892.167117,34966.234625,145500.000000,207771.890425,97477.090199,...,40959.765297,106486.263206,284477.948540,12863.405835,173498.950776,179045.849789,7120.000000,6993.487156,34200.000000,13363.589462,80292.560450,50629.904928,50162.327161,25540.977742,288109.541498,602640.532432,716909.983458,21094.681029,52580.293758,71712.702915,27250.000000,116500.000000,37382.252218,13302.602101,3315.840856,36843.778698,18350.000000,65811.316984,93901.814940,5660.000000,49100.000000,52210.680232,85453.401768,35962.332464,25566.959604,52974.745922,337500.039882,261600.000000,27102.876104,17037.310464
3,2021-11-04,27663.586512,35740.041537,59527.648427,141644.927351,97479.070447,16745.830367,46404.523354,84350.106097,97325.357224,701654.144381,50544.547211,231992.755028,33723.312446,55687.405674,17196.563155,2269.124072,28430.218331,5107.345932,335055.505310,51446.45

In [ ]:
mission29_1

,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,28585.166405,32700.000000,59216.443470,135000.000000,120768.837790,15498.504406,44149.671695,82633.395986,115715.571737,645195.755547,46475.763108,206619.321072,31456.503673,72987.876669,15036.988877,1858.336768,24619.386805,5034.255867,311469.287189,44098.302760,12850.000000,30190.010853,9722.464767,7703.252692,27173.659951,159446.555284,42762.411066,80608.723808,39246.388159,219645.554781,285952.396048,85144.759337,95052.784427,30912.391077,26271.631743,38450.000000,138686.877673,204792.351253,94909.428497,96782.767232,...,42556.741569,107984.654667,362478.721260,12800.364801,148786.939366,146500.000000,6209.537367,7216.061292,33014.114874,13430.988176,110239.227476,49428.098302,48961.445638,22320.850002,242441.203674,511828.214219,628547.525040,21376.906003,51516.920168,51294.957583,42353.868291,110421.594114,36616.916332,13164.461390,2956.299095,45396.044627,18324.579671,55256.345771,98302.358765,5651.109428,44536.421650,52005.114981,117124.155077,42195.317671,19793.928765,50405.692178,405509.155925,276313.632733,25554.798201,16815.830251
1,28601.787056,31850.000000,59131.400735,131000.000000,121061.572833,15503.168242,43989.415286,83268.751768,115670.699938,645644.473500,46515.224647,206719.748423,31465.435767,73081.039062,15064.208535,1860.337285,24628.821051,5034.096786,314354.252773,44056.208743,13100.000000,30089.785526,9724.729726,7708.041245,27198.179358,159621.768959,42780.721157,80717.270818,39238.268507,219564.358202,285909.661009,89451.899387,95136.545111,30894.538031,26153.448522,37100.000000,138821.493057,204783.804244,94924.385757,96799.861248,...,42803.177462,107976.317737,362951.901212,12844.684348,148857.452186,146000.000000,6218.099374,7212.642486,33012.164137,13428.364246,104283.603997,48199.087161,48847.479826,22303.572224,243396.757579,513097.444988,626481.105211,21334.384637,51408.257775,51327.863563,42507.916706,110301.935996,36725.475098,13251.809329,2943.393105,45417.839497,18259.658395,54566.174852,98063.445737,5652.783881,44498.173787,52168.875543,115645.046416,39563.742989,19652.844683,50378.196451,402076.855638,276249.957519,25531.758708,16795.472973
2,28698.676466,30950.000000,59046.358000,129500.000000,121354.307875,15507.832079,43829.158877,83361.220537,115625.828138,646093.191452,46554.686185,206820.175773,31474.367862,73174.201455,15091.428194,1862.337803,24638.255298,5033.937706,311166.681468,44014.114726,12900.000000,29989.560199,9726.994684,7712.829797,27222.698766,159796.982633,42799.031249,80825.817828,39230.148855,219483.161623,285866.925969,91640.782869,95220.305795,30876.684985,26035.265301,36250.000000,138956.108442,204775.257236,94939.343017,96816.955265,...,42516.473864,107967.980806,361201.837918,12780.339653,148927.965007,150500.000000,6226.661381,7209.223679,33010.213400,13425.740315,112243.968938,47959.274899,48733.514014,22286.294446,245718.263134,514366.675758,618545.702246,21291.863270,51299.595381,51360.769544,42661.965121,110182.277877,37284.011621,13226.588323,2930.487116,45439.634367,18273.705655,53876.003933,97824.532710,5654.458335,44459.925924,51961.913308,117147.277301,38729.021726,19511.760600,50350.700723,394428.461402,276186.282305,25447.476409,16775.115695
3,28113.072091,32387.765981,58961.315265,132116.611876,121647.042918,15512.495915,43668.902468,83585.660050,115580.956339,646541.909404,46594.147724,206920.603123,31483.299957,73267.363848,15118.647853,1864.338320,24647.689545,5033.778626,312610.198707

In [ ]:
#날짜별 주식 값 가져오기 
start_date = '20211129'
end_date = '20211203'
start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
print(Business_days)
data = Business_days.copy()
stock_list['종목코드'].values.sort()
for code in tqdm(stock_list['종목코드'].values):
    fdr_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(data,fdr_data, how='outer', on='Date')
    # data.Close = data.Close.ffill()
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)
columns = list(sample_submission.columns[1:])
columns = ['Day'] + [str(x).zfill(6) for x in columns]
data.columns = columns


        Date
0 2021-11-29
1 2021-11-30
2 2021-12-01
3 2021-12-02
4 2021-12-03


100%|██████████| 370/370 [05:16<00:00,  1.17it/s]


In [ ]:
data.drop("Day",axis=1,inplace=True)

In [ ]:
data

,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,31200,30300,58900,129000,110500,15200,43250,79200,116000,641000,45600,208000,30950,73500,14800,1845,24250,4870,285000,43250,12400,31700,9320,7600,26550,155000,40700,79600,38400,215500,284500,78000,94200,30050,25600,35000,136000,200500,93000,95000,...,38300,107880,344300,12550,147500,145500,6210,7000,31900,13305,102800,36250,47479,21750,259500,511000,618000,20200,49600,57000,46750,116000,34550,12700,2960,44750,18550,53000,95800,5600,43750,48750,100500,37100,19800,49100,369000,266300,24200,17650
1,31300,29000,57800,124000,108500,15150,42150,77800,114000,632000,44200,202000,29850,67000,14200,1800,23000,4790,275000,41250,11900,30900,9020,7500,26500,157500,40050,78100,37100,217500,273500,74700,90500,29000,24600,33650,131000,195500,90800,92700,...,37900,108225,344800,12275,146000,139000,6720,6450,31250,13050,98600,32800,47193,20800,246000,502000,598000,19050,48000,52000,49200,112000,31600,12600,2850,44000,17750,52600,95300,5440,43300,49250,97800,35550,18600,45500,364500,255800,23100,19500
2,31700,29400,57700,125000,112000,15500,42750,81200,116500,630000,45200,203000,30850,68100,15250,1830,23550,4795,296000,41800,12100,30900,9180,7580,27400,157500,41900,75800,37750,222000,279500,74400,92300,29350,25300,34100,131500,201000,93200,95200,...,38700,108470,341200,12590,143500,141500,7170,6580,31650,13400,96700,32300,47050,21350,294500,534000,622000,19400,49350,51600,49250,114000,32500,12450,2940,44100,17800,53400,95000,5590,48950,48700,98500,36050,18900,46200,352500,264200,23850,18650
3,32150,29550,60100,129000,110000,16150,43100,81600,120000,639000,47150,203500,31450,68400,15500,1820,23800,4865,301500,43550,12450,31500,9330,7610,27800,149000,42750,76700,40700,234500,286000,75100,94900,30450,26000,34100,130500,205500,95800,98600,...,39050,108540,317500,12785,150500,142500,7540,6880,31900,13565,91500,33500,45400,22800,291500,569000,618000,19950,49700,52700,45500,113000,33700,12500,2955,47200,17900,55000,97000,5590,51900,46250,95100,33800,18650,48550,330000,274700,25200,18050
4,32700,30600,60300,131000,108500,16400,44900,82500,118000,642000,48300,209000,31750,68800,15650,1865,24200,4880,313500,44400,12900,31750,9480,7700,28500,150000,42500,78900,40750,241000,286000,76500,100000,30500,26550,33850,133000,206000,95500,98700,...,40000,108300,329000,12880,151000,144500,8020,6920,35000,13580,95200,34000,45900,23100,317500,597000,674000,20250,51700,51800,45900,117000,33750,12750,3010,48400,18100,56600,99900,5650,51900,46800,95300,34500,19100,49000,354500,275900,25800,18150


In [ ]:
per =abs(mission29_1 -data)/mission29_1
per1= per.mean()
print(per.mean().sum()*100/370)
for i in range(370):
    total_list[i][1]=per1[i]

5.081580686468918


In [ ]:
mission1_5 = pd.read_csv('/content/drive/MyDrive/stock/mission.csv')
mission29_1 =ari_sb
final=pd.concat([mission1_5,mission29_1])
final["Day"]=["2021-11-01","2021-11-02","2021-11-03","2021-11-04","2021-11-05","2021-11-29","2021-11-30","2021-12-01","2021-12-02","2021-12-03"]
final.index= final["Day"]
final.drop("Day",axis=1,inplace=True)
final.reset_index(inplace=True)
final.to_csv("/content/drive/MyDrive/stock/final_mission2.csv", index=False)
final

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27479.842216,36138.095309,59798.209178,142016.422696,95225.613773,16713.587376,46891.672887,84056.764192,98222.613725,720762.814054,50361.126651,231319.857832,33663.202372,54667.818208,17079.226698,2257.003263,28298.159917,5106.851131,333177.000637,51600.000000,12987.616025,31551.517476,9434.864767,7680.000000,30391.464715,145654.518592,45864.846980,85261.828596,44461.530281,248169.299796,288500.000000,82277.460899,103123.833092,33466.540522,29123.987871,34984.926227,159500.000000,207767.217522,97422.884594,...,40294.094097,106518.185307,283622.808353,12967.648389,174143.743504,179832.868481,7270.000000,7002.958153,35600.000000,13369.888519,79976.672599,50466.661947,50396.668798,25548.855473,286913.279816,599051.747384,711695.030187,21189.389298,52805.779739,71454.758990,27200.000000,119000.000000,37042.570458,13267.648631,3283.739239,36169.435821,18950.000000,65449.124033,94461.628037,5700.000000,46400.000000,52248.763527,82314.402539,36026.351168,25819.107026,51853.261707,335831.067919,264200.000000,27109.877584,17078.431964
1,2021-11-02,27541.090315,36207.635430,59708.022261,141892.590915,96465.097047,16724.335040,46729.289709,84285.364662,97784.823214,719278.675955,50422.266838,231544.156897,33683.239063,55269.834105,17118.338850,2261.043533,28367.731617,5107.016065,333803.168861,50600.000000,12913.316972,31448.160079,9443.896530,7730.000000,30444.957615,145783.023262,47954.766425,85402.015516,44476.988401,248213.692321,284000.000000,82312.001469,103252.805055,33603.656818,29008.077494,34975.580426,158000.000000,207769.553974,97449.987396,...,41181.591587,106502.224256,284050.378446,12899.428591,173999.891121,179439.359135,7220.000000,6998.222654,35100.000000,13366.738991,80134.616525,50548.283437,50279.497980,25544.916607,287511.410657,600846.139908,714302.506823,21142.035163,52693.036748,71583.730952,26900.000000,118000.000000,37087.228199,13285.125366,3233.619442,36625.287961,18950.000000,66268.822658,94181.721488,5660.000000,49200.000000,52229.721880,84160.171715,35994.341816,25693.033315,52211.959579,336665.553900,263800.000000,27106.376844,17057.871214
2,2021-11-03,27602.338413,35963.989673,59617.835344,141768.759133,96891.585430,16735.082703,46566.906532,84293.033940,97999.656839,711611.180447,50483.407024,231768.455963,33703.275755,55239.146521,17157.451003,2265.083803,28447.954048,5107.180999,334429.337085,49600.000000,12839.017920,31344.802682,9387.676928,7850.000000,30325.218738,145911.527932,46983.363996,85542.202435,44492.446521,248258.084847,283500.000000,82346.542039,103381.777017,33452.259459,28892.167117,34966.234625,145500.000000,207771.890425,97477.090199,...,40959.765297,106486.263206,284477.948540,12863.405835,173498.950776,179045.849789,7120.000000,6993.487156,34200.000000,13363.589462,80292.560450,50629.904928,50162.327161,25540.977742,288109.541498,602640.532432,716909.983458,21094.681029,52580.293758,71712.702915,27250.000000,116500.000000,37382.252218,13302.602101,3315.840856,36843.778698,18350.000000,65811.316984,93901.814940,5660.000000,49100.000000,52210.680232,85453.401768,35962.332464,25566.959604,52974.745922,337500.039882,261600.000000,27102.876104,17037.310464
3,2021-11-04,27663.586512,35740.041537,59527.648427,141644.927351,97479.070447,16745.830367,46404.523354,84350.106097,97325.357224,701654.144381,50544.547211,231992.755028,33723.312446,55687.405674,17196.563155,2269.124072,28430.218331,5107.345932,335055.505310,51446.45